In [1]:
from math import exp
from math import pow
from math import sqrt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
from matplotlib.animation import FuncAnimation
from IPython import display
import numpy as np

%matplotlib qt

N=10
x = np.arange(-N,N,.1)
y = np.arange(-N,N,.1)
zz = 0
U_GT01=np.zeros((20*N,20*N))
xc=-1
for xx in x:
	xc+=1
	yc=-1
	for yy in y:
		yc+=1
		#-----------------------------------------------------------
		#
		# Teukolsky (quadrupole) waves
		#
		# This is a first attempt at coding up linearized quadrupole
		# waves.  As a first go, we will make some simplifying
		# assumptions.  In particular, we assume time symmetric
		# initial data.  We will assume an l=2, m=0 multipole.  We
		# will also work in transverse-traceless (TT) gauge.  (This
		# is the gauge that Teukolsky and Rinne work in.)  We also
		# work only with the even parity wave.
		#
		#-----------------------------------------------------------

		# We may eventually want a variety of options for the kinds
		# of waves that we want so we may as well define them here.

		# Choose the gauge in which we will work.  Choices include
		# TT gauge (0, default) and the "LATE" gauge of
		# Lorentz-Thorne-Abrahams-Evans (1)
		teukolsky_wave_gauge = 1

		# Choose the multipole order:  (l,m).  Choices include
		# quadrupole (l=2, default), octopole (l=3), and hexadecapole
		# (l=4) with all the possible m values (m=0, default).
		multipole_l = 2
		multipole_m = 0

		# Choose the polarization state.  Choices are either even
		# (0, default) or odd (1) parity waves.
		multipole_parity = 0

		# Choose the dynamics or time behavior of the initial data.
		# Choices include time symmetric (0, default), ingoing (1),
		# outgoing (2), time anti-symmetric (3) and general (4).
		id_dynamics_type = 0

		# Choose the functional form of the initial pulse.  Choices
		# include a C^k polynomial with compact support (0, default)
		# a Gaussian (1), a bump function (2), and Abrahams and Evans"
		# Hermite polynomial modulated Gaussian (3).
		id_functional_form = 1

		# The amplitude, (half-)width and center of the initial data:
		amp_teuk = .01
		width_teuk =1 #half width really width is twice this
		r_0 = 5
		# The order of differentiability of the C^k polynomials with
		# compact support.
		kk = 6

		if id_dynamics_type == 0: # time symmetric
			a_0 = 1.0
			b_0 = 0.0
		elif id_dynamics_type == 1:  # ingoing
			a_0 = 1.0
			b_0 = 1.0
		elif id_dynamics_type == 2:  # outgoing
			a_0 = 1.0
			b_0 = -1.0
		elif id_dynamics_type == 3: # time anti-symmetric
			a_0 = 0.0
			b_0 = 1.0
		else:
			print("this initial data choice is not yet allowed/n")

		pi = 3.1415926535897932384626433832795028841971693993751058209749445923078164062862089986

		#print( "starting the loop in initial data ... /n"

		rho_sqrd = xx*xx + yy*yy
		rho = sqrt( rho_sqrd )

		r_sqrd = rho_sqrd + zz*zz
		r = sqrt( r_sqrd )

		epsilon_sq = 1.0e-8    # maybe we can read in grid info later? 
		r_origin = sqrt( r_sqrd + epsilon_sq )

		# Away from the axis of symmetry
		if abs(xx) > 1.0e-7  or  abs(yy) > 1.0e-7:
			sin_theta = rho / r
			cos_theta = zz / r
			sin_2theta = 2.0 * sin_theta * cos_theta
			cos_2theta = pow(cos_theta,2)-pow(sin_theta,2)
			sin_3theta = ( 3.0 * sin_theta ) - ( 4.0 * pow(sin_theta,3) )
			cos_3theta = ( 4.0 * pow(cos_theta,3) )-( 3.0 * cos_theta )
			sin_4theta = cos_theta * (4.0 * sin_theta - 8.0 * pow(sin_theta,3))
			cos_4theta = 8.0 * pow(cos_theta,4) - 8.0 * pow(cos_theta,2) + 1.0
			sin_phi = yy / rho
			cos_phi = xx / rho
			sin_2phi = 2.0 * sin_phi * cos_phi
			cos_2phi = pow(cos_phi,2) - pow(sin_phi,2)
			sin_3phi = ( 3.0 * sin_phi ) - ( 4.0 * pow(sin_phi,3) )
			cos_3phi = ( 4.0 * pow(cos_phi,3) ) - ( 3.0 * cos_phi )
			sin_4phi = cos_phi * (4.0 * sin_phi - 8.0 * pow(sin_phi,3))
			cos_4phi = 8.0 * pow(cos_phi,4) - 8.0 * pow(cos_phi,2) + 1.0
		# On the axis of symmetry but away from the origin
		elif ( abs(xx) < 1.0e-7 and abs(yy) < 1.0e-7 ) and abs(zz) > 1.0e-7:
			sin_theta = rho / r
			cos_theta = zz / r
			sin_2theta = 2.0 * sin_theta * cos_theta
			cos_2theta = pow(cos_theta,2)-pow(sin_theta,2)
			sin_3theta = ( 3.0 * sin_theta ) - ( 4.0 * pow(sin_theta,3) )
			cos_3theta = ( 4.0 * pow(cos_theta,3) )-( 3.0 * cos_theta )
			sin_4theta = cos_theta * (4.0 * sin_theta - 8.0 * pow(sin_theta,3))
			cos_4theta = 8.0 * pow(cos_theta,4) - 8.0 * pow(cos_theta,2) + 1.0
			sin_phi = 0.0
			cos_phi = 0.0
			sin_2phi = 2.0 * sin_phi * cos_phi
			cos_2phi = pow(cos_phi,2) - pow(sin_phi,2)
			sin_3phi = ( 3.0 * sin_phi ) - ( 4.0 * pow(sin_phi,3) )
			cos_3phi = ( 4.0 * pow(cos_phi,3) ) - ( 3.0 * cos_phi )
			sin_4phi = cos_phi * (4.0 * sin_phi - 8.0 * pow(sin_phi,3))
			cos_4phi = 8.0 * pow(cos_phi,4) - 8.0 * pow(cos_phi,2) + 1.0
		# At the origin
		elif ( abs(xx) < 1.0e-7 and abs(yy) < 1.0e-7 ) and abs(zz) < 1.0e-7:
			sin_theta = 0.0
			cos_theta = 0.0
			sin_2theta = 2.0 * sin_theta * cos_theta
			cos_2theta = pow(cos_theta,2)-pow(sin_theta,2)
			sin_3theta = ( 3.0 * sin_theta ) - ( 4.0 * pow(sin_theta,3) )
			cos_3theta = ( 4.0 * pow(cos_theta,3) )-( 3.0 * cos_theta )
			sin_4theta = cos_theta * (4.0 * sin_theta - 8.0 * pow(sin_theta,3))
			cos_4theta = 8.0 * pow(cos_theta,4) - 8.0 * pow(cos_theta,2) + 1.0
			sin_phi = 0.0
			cos_phi = 0.0
			sin_2phi = 2.0*sin_phi*cos_phi
			cos_2phi = pow(cos_phi,2) - pow(sin_phi,2)
			sin_3phi = ( 3.0 * sin_phi ) - ( 4.0 * pow(sin_phi,3) )
			cos_3phi = ( 4.0 * pow(cos_phi,3) ) - ( 3.0 * cos_phi )
			sin_4phi = cos_phi * (4.0 * sin_phi - 8.0 * pow(sin_phi,3))
			cos_4phi = 8.0 * pow(cos_phi,4) - 8.0 * pow(cos_phi,2) + 1.0
		else:
			print(" problem in initial data /n")
			print( " x = " + xx) 
			print(" y = " + yy)
			print(" z = " + zz)
			print( " r = " + r)
			print( " rho = " +  rho)
		#######################################################################################/            
		#building the initial radial function
		#
		if id_functional_form == 0: # C^k polynomail with compact support
			aa = r_0 - width_teuk # leftmost zero of initial data
			bb = r_0 + width_teuk # rightmost zero of initial data

			if aa < 0.0 or bb < 0.0:
				print("initial data extends to r<0")	

			if r >= aa and r <= bb:
				f1 = pow((r - aa),(kk+1))
				f2 = pow((bb - r),(kk+1))

				df1 = (kk+1)*pow((r-aa),kk)
				df2 = - (kk+1)*pow((bb-r),kk)
			
				ddf1 = (kk+1)*kk*pow((r-aa),(kk-1))
				ddf2 = (kk+1)*kk*pow((bb-r),(kk-1))
			
				d3f1 = (kk+1)*kk*(kk-1)*pow((r-aa),(kk-2))
				d3f2 = - (kk+1)*kk*(kk-1)*pow((bb-r),(kk-2))
			
				d4f1 = (kk+1)*kk*(kk-1)*(kk-2)*pow((r-aa),(kk-3))
				d4f2 = (kk+1)*kk*(kk-1)*(kk-2)*pow((bb-r),(kk-3))
			
				d5f1 = (kk+1)*kk*(kk-1)*(kk-2)*(kk-3)*pow((r-aa),(kk-4))
				d5f2 = (kk+1)*kk*(kk-1)*(kk-2)*(kk-3)*pow((bb-r),(kk-4))

				d6f1 = (kk+1)*kk*(kk-1)*(kk-2)*(kk-3)*(kk-4)*pow((r-aa),(kk-5))
				d6f2 = (kk+1)*kk*(kk-1)*(kk-2)*(kk-3)*(kk-4)*pow((bb-r),(kk-5))
			
				f0 = f1 * f2
			
				df0 = df1*f2 + f1*df2
			
				ddf0 = ddf1*f2 + 2.0*df1*df2 + f1*ddf2
			
				d3f0 = d3f1*f2 + 3.0*ddf1*df2 + 3.0*df1*ddf2 + f1*d3f2 
			
				d4f0 = d4f1*f2 + 4.0 * d3f1*df2 + 6.0 * ddf1*ddf2 + 4.0 * df1*d3f2 + f1*d4f2
			
				d5f0 = d5f1*f2 + 5.0 * d4f1*df2 + 10.0 * d3f1*ddf2 + 10.0 * ddf1*d3f2 + 5.0 * df1*d4f2 + f1*d5f1
			
				d6f0 = d6f1*f2 + 6.0 * d5f1*df2 + 15.0 * d4f1*ddf2 + 20.0 * d3f1*d3f2 + 15.0 * ddf1*d4f2 +  6.0 * df1*d5f2 + f1*d6f2
			else:
				f0 = 0.0 
				df0 = 0.0 
				ddf0 = 0.0
				d3f0 = 0.0
				d4f0 = 0.0
				d5f0 = 0.0
				d6f0 = 0.0
		elif id_functional_form == 1: #Guassian
			f0 = exp( - pow(r-r_0,2) / width_teuk )
			df0 = ( -2.0 * (r-r_0) / width_teuk ) * f0
			ddf0 = ( (4.0 * pow(r-r_0, 2) - 2.0 * width_teuk) / pow(width_teuk,2) ) * f0
			d3f0 = ( (-8.0 * pow(r-r_0, 3) + 12.0 * (r-r_0) * width_teuk ) / pow(width_teuk,3) ) * f0
			d4f0 = ( (16.0 * pow(r-r_0, 4) -48.0 * pow(r-r_0,2) * width_teuk + 12.0 * pow(width_teuk,2) ) / pow(width_teuk,4) ) * f0
			d5f0 = (8.0*(r-r_0)*(-4.0 * pow(r-r_0,4) + 20.0 * pow(r-r_0,2) * width_teuk - 15.0 * pow(width_teuk,2) / pow(width_teuk,5) ) ) * f0
			d6f0 = (8.0 * (8.0 * pow(r-r_0,6) - 60.0 * pow(r-r_0,4) * width_teuk + 90.0 * pow(r-r_0,2) * pow(width_teuk,2) - 15.0 * pow(width_teuk,3) ) / pow(width_teuk,6) ) * f0
		########################################################################################
		#Setting up the radial functions

		if multipole_l == 2:
			if ( r > 1.0e-7 ):
				A_lm = amp_teuk * 0.5 * sqrt(21.0) * ( ( a_0 + b_0 ) * ddf0 / pow(r,3)
						+ ( a_0 - b_0 ) * ddf0 / pow(r,3)
						- 3.0 * ( a_0 + b_0 ) * df0 / pow(r,4)
						- 3.0 * ( a_0 - b_0 ) * df0 / pow(r,4)
						+ 3.0 * ( a_0 + b_0 ) * f0 / pow(r,5)
						+ 3.0 * ( a_0 - b_0 ) * f0 / pow(r,5)
					) 

				B_lm = amp_teuk * 0.5 * sqrt(7.0/12.0) * ( ( a_0 + b_0 ) * d3f0 / pow(r,2)
						+ ( a_0 - b_0 ) * d3f0 / pow(r,2)
						- 3.0 * ( a_0 + b_0 ) * ddf0 / pow(r,3)
						- 3.0 * ( a_0 - b_0 ) * ddf0 / pow(r,3)
						+ 6.0 * ( a_0 + b_0 ) * df0 / pow(r,4)
						+ 6.0 * ( a_0 - b_0 ) * df0 / pow(r,4)
						- 6.0 * ( a_0 + b_0 ) * f0 / pow(r,5)
						- 6.0 * ( a_0 - b_0 ) * f0 / pow(r,5)
					)

				C_lm = amp_teuk * 0.5 * 0.25 * sqrt(7.0/3.0) * ( ( a_0 + b_0 ) * d4f0 / r
						+ ( a_0 - b_0 ) * d4f0 / r
						- 2.0 * ( a_0 + b_0 ) * d3f0 / pow(r,2)
						- 2.0 * ( a_0 - b_0 ) * d3f0 / pow(r,2)
						+ 3.0 * ( a_0 + b_0 ) * ddf0 / pow(r,3)
						+ 3.0 * ( a_0 - b_0 ) * ddf0 / pow(r,3)
						- 3.0 * ( a_0 + b_0 ) * df0 / pow(r,4)
						- 3.0 * ( a_0 - b_0 ) * df0 / pow(r,4)
						+ 3.0 * ( a_0 + b_0 ) * f0 / pow(r,5)
						+ 3.0 * ( a_0 - b_0 ) * f0 / pow(r,5)
					)

				K_lm = 0.0
				
				L_lm = 0.0
			elif r < 1.0e-7:
				A_lm = amp_teuk * 0.5 * sqrt(21.0) * ( ( a_0 + b_0 ) * ddf0 / pow(r_origin,3)
						+ ( a_0 - b_0 ) * ddf0 / pow(r_origin,3)
						- 3.0 * ( a_0 + b_0 ) * df0 / pow(r_origin,4)
						- 3.0 * ( a_0 - b_0 ) * df0 / pow(r_origin,4)
						+ 3.0 * ( a_0 + b_0 ) * f0 / pow(r_origin,5)
						+ 3.0 * ( a_0 - b_0 ) * f0 / pow(r_origin,5)
					)

				B_lm = amp_teuk * 0.5 * sqrt(7.0/12.0) * ( ( a_0 + b_0 ) * d3f0 / pow(r_origin,2)
						+ ( a_0 - b_0 ) * d3f0 / pow(r_origin,2)
						- 3.0 * ( a_0 + b_0 ) * ddf0 / pow(r_origin,3)
						- 3.0 * ( a_0 - b_0 ) * ddf0 / pow(r_origin,3)
						+ 6.0 * ( a_0 + b_0 ) * df0 / pow(r_origin,4)
						+ 6.0 * ( a_0 - b_0 ) * df0 / pow(r_origin,4)
						- 6.0 * ( a_0 + b_0 ) * f0 / pow(r_origin,5)
						- 6.0 * ( a_0 - b_0 ) * f0 / pow(r_origin,5)
					)

				C_lm = amp_teuk * 0.5 * 0.25 * sqrt(7.0/3.0) * (         ( a_0 + b_0 ) * d4f0 / r_origin
						+       ( a_0 - b_0 ) * d4f0 / r_origin
						- 2.0 * ( a_0 + b_0 ) * d3f0 / pow(r_origin,2)
						- 2.0 * ( a_0 - b_0 ) * d3f0 / pow(r_origin,2)
						+ 3.0 * ( a_0 + b_0 ) * ddf0 / pow(r_origin,3)
						+ 3.0 * ( a_0 - b_0 ) * ddf0 / pow(r_origin,3)
						- 3.0 * ( a_0 + b_0 ) * df0 / pow(r_origin,4)
						- 3.0 * ( a_0 - b_0 ) * df0 / pow(r_origin,4)
						+ 3.0 * ( a_0 + b_0 ) * f0 / pow(r_origin,5)
						+ 3.0 * ( a_0 - b_0 ) * f0 / pow(r_origin,5)
					)

				K_lm = 0.0
				
				L_lm = 0.0
		elif multipole_l==3:
					
			if ( r > 1.0e-7 ):
				A_lm = amp_teuk * 0.5 * 3 * sqrt(10) * (          ( a_0 + b_0 ) * d3f0 / pow(r,3)
							+        ( a_0 - b_0 ) * d3f0 / pow(r,3)
							-  6.0 * ( a_0 + b_0 ) * ddf0 / pow(r,4)
							-  6.0 * ( a_0 - b_0 ) * ddf0 / pow(r,4)
							+ 15.0 * ( a_0 + b_0 ) * df0 / pow(r,5)
							+ 15.0 * ( a_0 - b_0 ) * df0 / pow(r,5)
						- 15.0 * ( a_0 + b_0 ) * f0 / pow(r,6)
						- 15.0 * ( a_0 - b_0 ) * f0 / pow(r,6)
					) 

				B_lm = amp_teuk * 0.5 * sqrt(5.0/8.0) * (         ( a_0 + b_0 ) * d4f0 / pow(r,2)
						+        ( a_0 - b_0 ) * d4f0 / pow(r,2)
						-  6.0 * ( a_0 + b_0 ) * d3f0 / pow(r,3)
						-  6.0 * ( a_0 - b_0 ) * d3f0 / pow(r,3)
						+ 21.0 * ( a_0 + b_0 ) * ddf0 / pow(r,4)
						+ 21.0 * ( a_0 - b_0 ) * ddf0 / pow(r,4)
						- 45.0 * ( a_0 + b_0 ) * df0 / pow(r,5)
						- 45.0 * ( a_0 - b_0 ) * df0 / pow(r,5)
						+ 45.0 * ( a_0 + b_0 ) * f0 / pow(r,6)
						+ 45.0 * ( a_0 - b_0 ) * f0 / pow(r,6)
					)

				C_lm = amp_teuk * 0.5 * 0.5 * sqrt(1.0/10.0)* (         ( a_0 + b_0 ) * d5f0 / r
						+        ( a_0 - b_0 ) * d5f0 / r
						-  5.0 * ( a_0 + b_0 ) * d4f0 / pow(r,2)
						-  5.0 * ( a_0 - b_0 ) * d4f0 / pow(r,2)
						+ 15.0 * ( a_0 + b_0 ) * d3f0 / pow(r,3)
						+ 15.0 * ( a_0 - b_0 ) * d3f0 / pow(r,3)
						- 30.0 * ( a_0 + b_0 ) * ddf0 / pow(r,4)
						- 30.0 * ( a_0 - b_0 ) * ddf0 / pow(r,4)
						+ 45.0 * ( a_0 + b_0 ) * df0 / pow(r,5)
						+ 45.0 * ( a_0 - b_0 ) * df0 / pow(r,5)
					- 45.0 * ( a_0 + b_0 ) * f0 / pow(r,6)
					- 45.0 * ( a_0 - b_0 ) * f0 / pow(r,6)
					)

				K_lm = 0.0
				
				L_lm = 0.0
			elif r < 1.0e-7 :
				A_lm = amp_teuk * 0.5 * 3 * sqrt(10) * (          ( a_0 + b_0 ) * d3f0 / pow(r_origin,3)
							+        ( a_0 - b_0 ) * d3f0 / pow(r_origin,3)
							-  6.0 * ( a_0 + b_0 ) * ddf0 / pow(r_origin,4)
							-  6.0 * ( a_0 - b_0 ) * ddf0 / pow(r_origin,4)
							+ 15.0 * ( a_0 + b_0 ) * df0 / pow(r_origin,5)
							+ 15.0 * ( a_0 - b_0 ) * df0 / pow(r_origin,5)
						- 15.0 * ( a_0 + b_0 ) * f0 / pow(r_origin,6)
						- 15.0 * ( a_0 - b_0 ) * f0 / pow(r_origin,6)
					) 

				B_lm = amp_teuk * 0.5 * sqrt(5.0/8.0) * (         ( a_0 + b_0 ) * d4f0 / pow(r_origin,2)
						+        ( a_0 - b_0 ) * d4f0 / pow(r_origin,2)
						-  6.0 * ( a_0 + b_0 ) * d3f0 / pow(r_origin,3)
						-  6.0 * ( a_0 - b_0 ) * d3f0 / pow(r_origin,3)
						+ 21.0 * ( a_0 + b_0 ) * ddf0 / pow(r_origin,4)
						+ 21.0 * ( a_0 - b_0 ) * ddf0 / pow(r_origin,4)
						- 45.0 * ( a_0 + b_0 ) * df0 / pow(r_origin,5)
						- 45.0 * ( a_0 - b_0 ) * df0 / pow(r_origin,5)
					+ 45.0 * ( a_0 + b_0 ) * f0 / pow(r_origin,6)
					+ 45.0 * ( a_0 - b_0 ) * f0 / pow(r_origin,6)
					)

				C_lm = amp_teuk * 0.5 * 0.5 * sqrt(1.0/10.0) * (         ( a_0 + b_0 ) * d5f0 / r_origin
						+        ( a_0 - b_0 ) * d5f0 / r_origin
						-  5.0 * ( a_0 + b_0 ) * d4f0 / pow(r_origin,2)
						-  5.0 * ( a_0 - b_0 ) * d4f0 / pow(r_origin,2)
						+ 15.0 * ( a_0 + b_0 ) * d3f0 / pow(r_origin,3)
						+ 15.0 * ( a_0 - b_0 ) * d3f0 / pow(r_origin,3)
						- 30.0 * ( a_0 + b_0 ) * ddf0 / pow(r_origin,4)
						- 30.0 * ( a_0 - b_0 ) * ddf0 / pow(r_origin,4)
						+ 45.0 * ( a_0 + b_0 ) * df0 / pow(r_origin,5)
						+ 45.0 * ( a_0 - b_0 ) * df0 / pow(r_origin,5)
					- 45.0 * ( a_0 + b_0 ) * f0 / pow(r_origin,6)
					- 45.0 * ( a_0 - b_0 ) * f0 / pow(r_origin,6)
					)

				K_lm = 0.0
				
				L_lm = 0.0
		elif multipole_l==4:
					
			if ( r > 1.0e-7 ):
				A_lm = amp_teuk * 0.5 * sqrt(770.0/3.0)  * (           ( a_0 + b_0 ) * d4f0 / pow(r,3)
							+         ( a_0 - b_0 ) * d4f0 / pow(r,3)
							-  10.0 * ( a_0 + b_0 ) * d3f0 / pow(r,4)
							-  10.0 * ( a_0 - b_0 ) * d3f0 / pow(r,4)
							+  45.0 * ( a_0 + b_0 ) * ddf0 / pow(r,5)
							+  45.0 * ( a_0 - b_0 ) * ddf0 / pow(r,5)
						- 105.0 * ( a_0 + b_0 ) * df0 / pow(r,6)
						- 105.0 * ( a_0 - b_0 ) * df0 / pow(r,6)
						+ 105.0 * ( a_0 + b_0 ) * f0 / pow(r,7)
						+ 105.0 * ( a_0 - b_0 ) * f0 / pow(r,7)
					) 

				B_lm = amp_teuk * 0.5 * sqrt(77.0/120.0) * (          ( a_0 + b_0 ) * d5f0 / pow(r,2)
						+         ( a_0 - b_0 ) * d5f0 / pow(r,2)
						-  10.0 * ( a_0 + b_0 ) * d4f0 / pow(r,3)
						-  10.0 * ( a_0 - b_0 ) * d4f0 / pow(r,3)
						+  55.0 * ( a_0 + b_0 ) * d3f0 / pow(r,4)
						+  55.0 * ( a_0 - b_0 ) * d3f0 / pow(r,4)
						- 195.0 * ( a_0 + b_0 ) * ddf0 / pow(r,5)
						- 195.0 * ( a_0 - b_0 ) * ddf0 / pow(r,5)
					+ 420.0 * ( a_0 + b_0 ) * df0 / pow(r,6)
					+ 420.0 * ( a_0 - b_0 ) * df0 / pow(r,6)
					- 420.0 * ( a_0 + b_0 ) * f0 / pow(r,7)
					- 420.0 * ( a_0 - b_0 ) * f0 / pow(r,7)
					)

				C_lm = amp_teuk * 0.5 * ( 1.0/18.0 ) * sqrt(77.0/30.0) * (         ( a_0 + b_0 ) * d6f0 / r
						+        ( a_0 - b_0 ) * d6f0 / r
						-   9.0 * ( a_0 + b_0 ) * d5f0 / pow(r,2)
						-   9.0 * ( a_0 - b_0 ) * d5f0 / pow(r,2)
						+  45.0 * ( a_0 + b_0 ) * d4f0 / pow(r,3)
						+  45.0 * ( a_0 - b_0 ) * d4f0 / pow(r,3)
						- 150.0 * ( a_0 + b_0 ) * d3f0 / pow(r,4)
						- 150.0 * ( a_0 - b_0 ) * d3f0 / pow(r,4)
						+ 360.0 * ( a_0 + b_0 ) * ddf0 / pow(r,5)
						+ 360.0 * ( a_0 - b_0 ) * ddf0 / pow(r,5)
					- 630.0 * ( a_0 + b_0 ) * f0 / pow(r,6)
					- 630.0 * ( a_0 - b_0 ) * f0 / pow(r,6)
					+ 630.0 * ( a_0 + b_0 ) * f0 / pow(r,7)
					+ 640.0 * ( a_0 - b_0 ) * f0 / pow(r,7)
					)

				K_lm = 0.0
				
				L_lm = 0.0
			elif r < 1.0e-7:
				A_lm = amp_teuk * 0.5 * sqrt(770.0/3.0) * (           ( a_0 + b_0 ) * d4f0 / pow(r_origin,3)
							+         ( a_0 - b_0 ) * d4f0 / pow(r_origin,3)
							-  10.0 * ( a_0 + b_0 ) * d3f0 / pow(r_origin,4)
							-  10.0 * ( a_0 - b_0 ) * d3f0 / pow(r_origin,4)
							+  45.0 * ( a_0 + b_0 ) * ddf0 / pow(r_origin,5)
							+  45.0 * ( a_0 - b_0 ) * ddf0 / pow(r_origin,5)
						- 105.0 * ( a_0 + b_0 ) * df0 / pow(r_origin,6)
						- 105.0 * ( a_0 - b_0 ) * df0 / pow(r_origin,6)
						+ 105.0 * ( a_0 + b_0 ) * f0 / pow(r_origin,7)
						+ 105.0 * ( a_0 - b_0 ) * f0 / pow(r_origin,7)
					) 

				B_lm = amp_teuk * 0.5 * sqrt(77.0/120.0) * (          ( a_0 + b_0 ) * d5f0 / pow(r_origin,2)
						+         ( a_0 - b_0 ) * d5f0 / pow(r_origin,2)
						-  10.0 * ( a_0 + b_0 ) * d4f0 / pow(r_origin,3)
						-  10.0 * ( a_0 - b_0 ) * d4f0 / pow(r_origin,3)
						+  55.0 * ( a_0 + b_0 ) * d3f0 / pow(r_origin,4)
						+  55.0 * ( a_0 - b_0 ) * d3f0 / pow(r_origin,4)
						- 195.0 * ( a_0 + b_0 ) * ddf0 / pow(r_origin,5)
						- 195.0 * ( a_0 - b_0 ) * ddf0 / pow(r_origin,5)
					+ 420.0 * ( a_0 + b_0 ) * df0 / pow(r_origin,6)
					+ 420.0 * ( a_0 - b_0 ) * df0 / pow(r_origin,6)
					- 420.0 * ( a_0 + b_0 ) * f0 / pow(r_origin,7)
					- 420.0 * ( a_0 - b_0 ) * f0 / pow(r_origin,7)
					)

				C_lm = amp_teuk * 0.5 * ( 1.0/18.0 ) * sqrt(77.0/30.0) * (         ( a_0 + b_0 ) * d6f0 / r_origin
						+        ( a_0 - b_0 ) * d6f0 / r_origin
						-   9.0 * ( a_0 + b_0 ) * d5f0 / pow(r_origin,2)
						-   9.0 * ( a_0 - b_0 ) * d5f0 / pow(r_origin,2)
						+  45.0 * ( a_0 + b_0 ) * d4f0 / pow(r_origin,3)
						+  45.0 * ( a_0 - b_0 ) * d4f0 / pow(r_origin,3)
						- 150.0 * ( a_0 + b_0 ) * d3f0 / pow(r_origin,4)
						- 150.0 * ( a_0 - b_0 ) * d3f0 / pow(r_origin,4)
						+ 360.0 * ( a_0 + b_0 ) * ddf0 / pow(r_origin,5)
						+ 360.0 * ( a_0 - b_0 ) * ddf0 / pow(r_origin,5)
					- 630.0 * ( a_0 + b_0 ) * f0 / pow(r_origin,6)
					- 630.0 * ( a_0 - b_0 ) * f0 / pow(r_origin,6)
					+ 630.0 * ( a_0 + b_0 ) * f0 / pow(r_origin,7)
					+ 640.0 * ( a_0 - b_0 ) * f0 / pow(r_origin,7)
					)

				K_lm = 0.0
				
				L_lm = 0.0



		# Now for the angular functions for l=2 
		#####################################/
		# Starting with Y_20
		#
		Y20  = ( 1.0/4.0 ) * sqrt(5.0/pi) * ( 3.0 * pow(cos_theta,2) - 1.0 )
		Y21  = ( 1.0/2.0 ) * sqrt(15.0/pi) * sin_theta * cos_theta * cos_phi
		Y2n1 = ( 1.0/2.0 ) * sqrt(15.0/pi) * sin_theta * cos_theta * sin_phi
		Y22  = ( 1.0/4.0 ) * sqrt(15.0/pi) * pow(sin_theta,2) * cos_2phi
		Y2n2 = ( 1.0/4.0 ) * sqrt(15.0/pi) * pow(sin_theta,2) * sin_2phi

		# Y_2m,theta
		#
		Y20_theta  = - ( 3.0/2.0 ) * sqrt(5.0/pi) * sin_theta * cos_theta 
		Y21_theta  =   ( 1.0/2.0 ) * sqrt( 15.0/pi) * cos_2theta * cos_phi
		Y2n1_theta =   ( 1.0/2.0 ) * sqrt( 15.0/pi) * cos_2theta * sin_phi
		Y22_theta  =   ( 1.0/2.0 ) * sqrt( 15.0/pi) * sin_theta * cos_theta * cos_2phi
		Y2n2_theta =   ( 1.0/2.0 ) * sqrt( 15.0/pi) * sin_theta * cos_theta * sin_2phi

		# Y_2m,phi
		#
		Y20_phi  =   0.0
		Y21_phi  = - ( 1.0/2.0 ) * sqrt(15.0/pi) * cos_theta * sin_phi
		Y2n1_phi =   ( 1.0/2.0 ) * sqrt(15.0/pi) * cos_theta * cos_phi
		Y22_phi  = - ( 1.0/2.0 ) * sqrt(15.0/pi) * sin_theta * sin_2phi
		Y2n2_phi =   ( 1.0/2.0 ) * sqrt(15.0/pi) * sin_theta * cos_2phi

		# Y_2m,theta,theta + .5 l ( l + 1 ) Y_2m
		#
		Y20_thth  =   ( 3.0/4.0 ) * sqrt(5.0/pi) * pow(sin_theta,2)
		Y21_thth  = - ( 1.0/2.0 ) * sqrt(15.0/pi) * sin_theta * cos_theta * cos_phi
		Y2n1_thth = - ( 1.0/2.0 ) * sqrt(15.0/pi) * sin_theta * cos_theta * sin_phi
		Y22_thth =    ( 1.0/8.0 ) * sqrt(15.0/pi) * ( 3.0 + cos_2theta ) * cos_2phi
		Y2n2_thth =   ( 1.0/8.0 ) * sqrt(15.0/pi) * ( 3.0 + cos_2theta ) * sin_2phi

		# Y_2m,theta,phi - cot_theta Y-2m,phi
		#
		Y20_thphi  =   0.0
		Y21_thphi  =   ( 1.0/2.0 ) * sqrt(15.0/pi) * sin_theta * sin_phi
		Y2n1_thphi = - ( 1.0/2.0 ) * sqrt(15.0/pi) * sin_theta * cos_phi
		Y22_thphi  = - ( 1.0/2.0 ) * sqrt(15.0/pi) * cos_theta * sin_2phi
		Y2n2_thphi =   ( 1.0/2.0 ) * sqrt(15.0/pi) * cos_theta * cos_2phi

			
		#l=3 Angular Functions
		##################################/
		#
		# Y_3m
		Y30  = ( 1.0/8.0 ) * sqrt(7.0/pi) * cos_theta * ( 5.0 * cos_2theta - 1.0 )
		Y31  = ( 1.0/16.0 ) * sqrt(21.0/2.0/pi) * ( sin_theta + 5.0 * sin_3theta ) * cos_phi
		Y3n1 = ( 1.0/16.0 ) * sqrt(21.0/2.0/pi) * ( sin_theta + 5.0 * sin_3theta ) * sin_phi
		Y32  = ( 1.0/4.0 ) * sqrt(105.0/pi) * pow(sin_theta,2) * cos_theta * cos_2phi
		Y3n2 = ( 1.0/4.0 ) * sqrt(105.0/pi) * pow(sin_theta,2) * cos_theta * sin_2phi
		Y33  = ( 1.0/4.0 ) * sqrt(35.0/2.0/pi) * pow(sin_theta,3) * cos_3phi
		Y3n3 = ( 1.0/4.0 ) * sqrt(35.0/2.0/pi) * pow(sin_theta,3) * sin_3phi

		# Y_3m,theta
		#
		Y30_theta  = - ( 3.0/8.0 ) * sqrt(7.0/pi) * ( 3.0 + 5.0 * cos_2theta ) * sin_theta
		Y31_theta  =   ( 1.0/16.0 ) * sqrt(21.0/2.0/pi) * (cos_theta + 15.0 * cos_3theta ) * cos_phi
		Y3n1_theta =   ( 1.0/16.0 ) * sqrt(21.0/2.0/pi) * (cos_theta + 15.0 * cos_3theta ) * sin_phi
		Y32_theta  =   ( 1.0/8.0 ) * sqrt(105.0/pi) * sin_theta * ( 1.0 + 3.0 * cos_2theta ) * cos_2phi
		Y3n2_theta =   ( 1.0/8.0 ) * sqrt(105.0/pi) * sin_theta * ( 1.0 + 3.0 * cos_2theta ) * sin_2phi
		Y33_theta  =   ( 3.0/4.0 ) * sqrt(35.0/2.0/pi) * cos_theta * pow(sin_theta,2) * cos_3phi
		Y3n3_theta =   ( 3.0/4.0 ) * sqrt(35.0/2.0/pi) * cos_theta * pow(sin_theta,2) * sin_3phi

		# Y_3m,phi
		#
		Y30_phi  =   0.0
		Y31_phi  = - ( 1.0/8.0 ) * sqrt(21.0/2.0/pi) * ( 3.0 + 5.0 * cos_2theta ) * sin_phi
		Y3n1_phi =   ( 1.0/8.0 ) * sqrt(21.0/2.0/pi) * ( 3.0 + 5.0 * cos_2theta ) * cos_phi
		Y32_phi  = - ( 1.0/4.0 ) * sqrt(105.0/pi) * sin_2theta * sin_2phi
		Y3n2_phi =   ( 1.0/4.0 ) * sqrt(105.0/pi) * sin_2theta * cos_2phi
		Y33_phi  = - ( 3.0/4.0 ) * sqrt(35.0/2.0/pi) * pow(sin_theta,2) * sin_3phi
		Y3n3_phi =   ( 3.0/4.0 ) * sqrt(35.0/2.0/pi) * pow(sin_theta,2) * cos_3phi

		# Y_3m,theta,theta + .5 l ( l+1 ) Y_3m
		# 
		Y30_thth  = ( 15.0/4.0 ) * sqrt(7.0/pi) * cos_theta * pow(sin_theta,2)
		Y31_thth  = ( 5.0/16.0 ) * sqrt(21.0/2.0/pi) * ( sin_theta - 3.0 * sin_3theta ) * cos_phi
		Y3n1_thth = ( 5.0/16.0 ) * sqrt(21.0/2.0/pi) * ( sin_theta - 3.0 * sin_3theta ) * sin_phi
		Y32_thth  = ( 1.0/16.0 ) * sqrt(105.0/pi) * ( 5.0 * cos_theta + 3.0 * cos_3theta ) * cos_2phi
		Y3n2_thth = ( 1.0/16.0 ) * sqrt(105.0/pi) * ( 5.0 * cos_theta + 3.0 * cos_3theta ) * sin_2phi
		Y33_thth  = ( 3.0/16.0 ) * sqrt(35.0/2.0/pi) * ( 5.0 * sin_theta + sin_3theta ) * cos_3phi
		Y3n3_thth  = ( 3.0/16.0 ) * sqrt(35.0/2.0/pi) * ( 5.0 * sin_theta + sin_3theta ) * sin_3phi

		# Y_3m,theta,phi - cot_theta Y_3m_phi
		#
		Y30_thphi  =   0.0
		Y31_thphi  =   ( 5.0/4.0 ) * sqrt(21.0/2.0/pi) * sin_2theta * sin_phi
		Y3n1_thphi = - ( 5.0/4.0 ) * sqrt(21.0/2.0/pi) * sin_2theta * cos_phi
		Y32_thphi  = - ( 1.0/2.0 ) * sqrt(105.0/pi) * cos_2theta * sin_2phi
		Y3n2_thphi =   ( 1.0/2.0 ) * sqrt(105.0/pi) * cos_2theta * cos_2phi
		Y33_thphi  = - ( 3.0/4.0 ) * sqrt(35.0/2.0/pi) * sin_2theta * sin_3phi
		Y3n3_thphi =   ( 3.0/4.0 ) * sqrt(35.0/2.0/pi) * sin_2theta * cos_3phi  


		# l=4 angular functions
		############################################/
		#
		#Y_4m
		#
		Y40  = ( 3.0/128.0 ) * sqrt( 1.0/pi ) * (9.0 + 20.0 * cos_2theta + 35.0 * cos_4theta)
		Y41  = ( 3.0/32.0 ) * sqrt( 5.0/2.0/pi ) * ( 2.0 * sin_2theta + 7.0 * sin_4theta ) * cos_phi
		Y4n1 = ( 3.0/32.0 ) * sqrt( 5.0/2.0/pi ) * ( 2.0 * sin_2theta + 7.0 * sin_4theta ) * sin_phi
		Y42  = ( 3.0/16.0 ) * sqrt( 5.0/pi ) * ( 5.0 + 7.0 * cos_2theta ) * pow(sin_theta,2) * cos_2phi
		Y4n2 = ( 3.0/16.0 ) * sqrt( 5.0/pi ) * ( 5.0 + 7.0 * cos_2theta ) * pow(sin_theta,2) * sin_2phi
		Y43  = ( 3.0/4.0 ) * sqrt( 35.0/2/pi ) * cos_theta * pow(sin_theta,3) * cos_3phi
		Y4n3 = ( 3.0/4.0 ) * sqrt( 35.0/2/pi ) * cos_theta * pow(sin_theta,3) * sin_3phi
		Y44  = ( 3.0/16.0 ) * sqrt( 35.0/pi ) * pow(sin_theta,4) * cos_4phi
		Y4n4 = ( 3.0/16.0 ) * sqrt( 35.0/pi ) * pow(sin_theta,4) * sin_4phi

		#Y_4m,theta
		#
		Y40_theta  = ( 15.0/4.0 ) * sqrt( 1.0/pi ) * ( 3.0 - 7.0 * pow(cos_theta,2) ) * sin_theta
		Y41_theta  = ( 3.0/8.0 ) * sqrt( 5.0/2.0/pi ) * (cos_2theta + 7.0 * cos_4theta) * cos_phi
		Y4n1_theta = ( 3.0/8.0 ) * sqrt( 5.0/2.0/pi ) * (cos_2theta + 7.0 * cos_4theta) * sin_phi
		Y42_theta  = ( 3.0/16.0 ) * sqrt( 5.0/pi ) * ( -2.0 * sin_2theta + 7.0 * sin_4theta ) * cos_2phi
		Y4n2_theta = ( 3.0/16.0 ) * sqrt( 5.0/pi ) * ( -2.0 * sin_2theta + 7.0 * sin_4theta ) * sin_2phi
		Y43_theta  = ( 3.0/4.0 ) * sqrt( 35.0/2.0/pi ) * (1.0 + 2.0 * cos_2theta) * pow(sin_theta,2) * cos_3phi 
		Y4n3_theta = ( 3.0/4.0 ) * sqrt( 35.0/2.0/pi ) * (1.0 + 2.0 * cos_2theta) * pow(sin_theta,2) * sin_3phi
		Y44_theta  = ( 3.0/4.0 ) * sqrt( 35.0/pi ) * cos_theta * pow(sin_theta,3) * cos_4phi
		Y4n4_theta = ( 3.0/4.0 ) * sqrt( 35.0/pi ) * cos_theta * pow(sin_theta,3) * sin_4phi

		#Y_4m,phi
		#
		Y40_phi  =   0.0
		Y41_phi  = - ( 3.0/16.0 ) * sqrt( 5.0/2.0/pi ) * ( 9.0 * cos_theta + 7.0 * cos_3theta ) * sin_phi
		Y4n1_phi =   ( 3.0/16.0 ) * sqrt( 5.0/2.0/pi ) * ( 9.0 * cos_theta + 7.0 * cos_3theta ) * cos_phi
		Y42_phi  = - ( 3.0/8.0 ) * sqrt( 5.0/pi ) * ( 5.0 + 7.0 * cos_2theta ) * sin_theta * sin_2phi
		Y4n2_phi =   ( 3.0/8.0 ) * sqrt( 5.0/pi ) * ( 5.0 + 7.0 * cos_2theta ) * sin_theta * cos_2phi
		Y43_phi  = - ( 9.0/4.0 ) * sqrt( 35.0/2.0/pi ) * cos_theta * pow(sin_theta,2) * sin_3phi
		Y4n3_phi =   ( 9.0/4.0 ) * sqrt( 35.0/2.0/pi ) * cos_theta * pow(sin_theta,2) * cos_3phi
		Y44_phi  = - ( 3.0/4.0 ) * sqrt( 35.0/pi ) * pow(sin_theta,3) * sin_4phi
		Y4n4_phi =   ( 3.0/4.0 ) * sqrt( 35.0/pi ) * pow(sin_theta,3) * cos_4phi

		#Y_4m,theta,theta + .5 l ( l+1 ) Y_4m
		#
		Y40_thth  = ( 45.0/16.0 ) * sqrt( 1.0/pi ) * ( 5.0 + 7.0 * cos_2theta ) * pow(sin_theta,2)
		Y41_thth  = ( 9.0/16.0 ) * sqrt( 5.0/2.0/pi ) * ( 2.0 * sin_2theta - 7.0 * sin_4theta ) * cos_phi
		Y4n1_thth = ( 9.0/16.0 ) * sqrt( 5.0/2.0/pi ) * ( 2.0 * sin_2theta - 7.0 * sin_4theta ) * sin_phi
		Y42_thth  = ( 9.0/32.0 ) * sqrt( 5.0/pi ) * ( 5.0 + 4.0 * cos_2theta + 7.0 * cos_4theta ) * cos_2phi
		Y4n2_thth = ( 9.0/32.0 ) * sqrt( 5.0/pi ) * ( 5.0 + 4.0 * cos_2theta + 7.0 * cos_4theta ) * sin_2phi
		Y43_thth  = ( 9.0/2.0 ) * sqrt( 35.0/2.0/pi ) * pow(cos_theta,3) * sin_theta * cos_3phi
		Y4n3_thth = ( 9.0/2.0 ) * sqrt( 35.0/2.0/pi ) * pow(cos_theta,3) * sin_theta * sin_3phi
		Y44_thth  = ( 9.0/16.0 ) * sqrt( 35.0/pi ) * ( 3.0 + cos_2theta ) * pow(sin_theta,2) * cos_4phi
		Y4n4_thth = ( 9.0/16.0 ) * sqrt( 35.0/pi ) * ( 3.0 + cos_2theta ) * pow(sin_theta,2) * sin_4phi

		#Y_4m,theta,phi - cot_theta Y_4m,phi
		#
		Y40_thphi  =   0.0
		Y41_thphi  =   ( 9.0/8.0 ) * sqrt( 5.0/2.0/pi ) * ( 5.0 + 7.0 * cos_2theta ) * sin_theta * sin_phi
		Y4n1_thphi = - ( 9.0/8.0 ) * sqrt( 5.0/2.0/pi ) * ( 5.0 + 7.0 * cos_2theta ) * sin_theta * cos_phi
		Y42_thphi  = - ( 9.0/16.0 ) * sqrt( 5.0/pi ) * ( cos_theta + 7.0 * cos_3theta ) * sin_2phi
		Y4n2_thphi =   ( 9.0/16.0 ) * sqrt( 5.0/pi ) * ( cos_theta + 7.0 * cos_3theta ) * cos_2phi
		Y43_thphi  = - ( 9.0/8.0 ) * sqrt( 35.0/2.0/pi ) * (1.0 + 3.0 * cos_2theta ) * sin_theta * sin_3phi
		Y4n3_thphi =   ( 9.0/8.0 ) * sqrt( 35.0/2.0/pi ) * (1.0 + 3.0 * cos_2theta ) * sin_theta * cos_3phi
		Y44_thphi  = - ( 9.0/4.0 ) * sqrt( 35.0/pi ) * cos_theta * pow(sin_theta,2) * sin_4phi
		Y4n4_thphi =   ( 9.0/4.0 ) * sqrt( 35.0/pi ) * cos_theta * pow(sin_theta,2) * cos_4phi

		############################################
		if multipole_l == 2:
			if multipole_m == 0:
				Ylm = Y20
				Ylm_theta = Y20_theta
				Ylm_thth = Y20_thth
				Ylm_thphi = Y20_thphi
				Ylm_phi = Y20_phi
			elif multipole_m == 1:
				Ylm = Y21
				Ylm_theta = Y21_theta
				Ylm_thth = Y21_thth
				Ylm_thphi = Y21_thphi
				Ylm_phi = Y21_phi	
			elif multipole_m == -1:
				Ylm = Y2n1
				Ylm_theta = Y2n1_theta
				Ylm_thth = Y2n1_thth
				Ylm_thphi = Y2n1_thphi
				Ylm_phi = Y2n1_phi	
			elif multipole_m == 2:
				Ylm = Y22
				Ylm_theta = Y22_theta
				Ylm_thth = Y22_thth
				Ylm_thphi = Y22_thphi
				Ylm_phi = Y22_phi
			elif multipole_m == -2:
				Ylm = Y2n2
				Ylm_theta = Y2n2_theta
				Ylm_thth = Y2n2_thth
				Ylm_thphi = Y2n2_thphi
				Ylm_phi = Y2n2_phi
			else:
				print("unnallowed m for l = 2/n")

		elif multipole_l == 3:
			if multipole_m == 0:
				Ylm = Y30
				Ylm_theta = Y30_theta
				Ylm_thth = Y30_thth
				Ylm_thphi = Y30_thphi
				Ylm_phi = Y30_phi
			elif multipole_m == 1:
				Ylm = Y31
				Ylm_theta = Y31_theta
				Ylm_thth = Y31_thth
				Ylm_thphi = Y31_thphi
				Ylm_phi = Y31_phi	
			elif multipole_m == -1:
				Ylm = Y3n1
				Ylm_theta = Y3n1_theta
				Ylm_thth = Y3n1_thth
				Ylm_thphi = Y3n1_thphi
				Ylm_phi = Y3n1_phi	
			elif multipole_m == 2:
				Ylm = Y32
				Ylm_theta = Y32_theta
				Ylm_thth = Y32_thth
				Ylm_thphi = Y32_thphi
				Ylm_phi = Y32_phi
			elif (multipole_m == -2):
				Ylm = Y3n2
				Ylm_theta = Y3n2_theta
				Ylm_thth = Y3n2_thth
				Ylm_thphi = Y3n2_thphi
				Ylm_phi = Y3n2_phi
			elif multipole_m == 3:
				Ylm = Y33
				Ylm_theta = Y33_theta
				Ylm_thth = Y33_thth
				Ylm_thphi = Y33_thphi
				Ylm_phi = Y33_phi
			elif multipole_m == -3:
				Ylm = Y3n3
				Ylm_theta = Y3n3_theta
				Ylm_thth = Y3n3_thth
				Ylm_thphi = Y3n3_thphi
				Ylm_phi = Y3n3_phi
		
			else:
				print("unnallowed m for l = 3/n")


		elif multipole_l == 4:
			if multipole_m == 0:
				Ylm = Y40
				Ylm_theta = Y40_theta
				Ylm_thth = Y40_thth
				Ylm_thphi = Y40_thphi
				Ylm_phi = Y40_phi
			elif multipole_m == 1:
				Ylm = Y41
				Ylm_theta = Y41_theta
				Ylm_thth = Y41_thth
				Ylm_thphi = Y41_thphi
				Ylm_phi = Y41_phi	
			elif multipole_m == -1:
				Ylm = Y4n1
				Ylm_theta = Y4n1_theta
				Ylm_thth = Y4n1_thth
				Ylm_thphi = Y4n1_thphi
				Ylm_phi = Y4n1_phi	
			elif multipole_m == 2:
				Ylm = Y42
				Ylm_theta = Y42_theta
				Ylm_thth = Y42_thth
				Ylm_thphi = Y42_thphi
				Ylm_phi = Y42_phi
			elif multipole_m == -2:
				Ylm = Y4n2
				Ylm_theta = Y4n2_theta
				Ylm_thth = Y4n2_thth
				Ylm_thphi = Y4n2_thphi
				Ylm_phi = Y4n2_phi
			elif multipole_m == 3:
				Ylm = Y43
				Ylm_theta = Y43_theta
				Ylm_thth = Y43_thth
				Ylm_thphi = Y43_thphi
				Ylm_phi = Y43_phi
			elif multipole_m == -3:
				Ylm = Y4n3
				Ylm_theta = Y4n3_theta
				Ylm_thth = Y4n3_thth
				Ylm_thphi = Y4n3_thphi
				Ylm_phi = Y4n3_phi
			elif multipole_m == 4:
				Ylm = Y44
				Ylm_theta = Y44_theta
				Ylm_thth = Y44_thth
				Ylm_thphi = Y44_thphi
				Ylm_phi = Y44_phi
			elif multipole_m == -4:
				Ylm = Y4n4
				Ylm_theta = Y4n4_theta
				Ylm_thth = Y4n4_thth
				Ylm_thphi = Y4n4_thphi
				Ylm_phi = Y4n4_phi

			else:
				print("unnallowed m for l = 4/n")

		########################################
		#Now put it all together to get the perturbed metric
		h_bare_rr   = A_lm * Ylm
		h_bare_rth  = B_lm * Ylm_theta - K_lm * Ylm_phi
		h_bare_rph  = B_lm * Ylm_phi + K_lm * Ylm_theta
		h_bare_thth =  - 0.5 * A_lm * Ylm + C_lm * Ylm_thth - L_lm * Ylm_thphi
		h_bare_thph = C_lm * Ylm_thphi + L_lm * Ylm_thth
		h_bare_phph = -(  0.5 * A_lm * Ylm + C_lm * Ylm_thth - L_lm * Ylm_thphi)

		g_bare_rr   = 1.0 + h_bare_rr
		g_bare_rth  = h_bare_rth
		g_bare_rph  = h_bare_rph
		g_bare_thth = 1.0 + h_bare_thth
		g_bare_thph = h_bare_thph
		g_bare_phph = 1.0 + h_bare_phph

		g_rr   =                           g_bare_rr
		g_rth  = r                       * g_bare_rth
		g_rph  = r * sin_theta           * g_bare_rph
		g_thth = r*r                     * g_bare_thth
		g_thph = r*r*sin_theta           * g_bare_thph
		g_phph = r*r*sin_theta*sin_theta * g_bare_phph



		# We now have to transform our metric from spherical coordinates
		# to Cartesian coordinates.  But we have to pay particular attention
		# to the axis and the origin of coordinates.
		if rho > 1.0e-7: # everywhere but the axis ...
			drdx = xx / r
			drdy = yy / r
			drdz = zz / r

			dthdx = xx * zz / ( r*r*rho )
			dthdy = yy * zz / ( r*r*rho )
			dthdz = - rho /( r*r )

			dphdx = - yy / ( rho_sqrd )
			dphdy =   xx / ( rho_sqrd )
			dphdz = 0.0

			g_xx =    drdx *  drdx * g_rr + dthdx * dthdx * g_thth + dphdx * dphdx * g_phph + 2.0 * (    drdx * dthdx * g_rth +  drdx * dphdx * g_rph + dthdx * dphdx * g_thph  )
			g_xy =    drdx *  drdy * g_rr + dthdx * dthdy * g_thth + dphdx * dphdy * g_phph + (  drdx*dthdy +  drdy*dthdx ) * g_rth + (  drdx*dphdy +  drdy*dphdx ) * g_rph + ( dthdx*dphdy + dthdy*dphdx ) * g_thph
			g_xz =   drdx *  drdz * g_rr + dthdx * dthdz * g_thth + dphdx * dphdz * g_phph + (  drdx*dthdz +  drdz*dthdx ) * g_rth + (  drdx*dphdz +  drdz*dphdx ) * g_rph + ( dthdx*dphdz + dthdz*dphdx ) * g_thph
			g_yy =  drdy *  drdy * g_rr + dthdy * dthdy * g_thth + dphdy * dphdy * g_phph + 2.0 * (    drdy * dthdy * g_rth +  drdy * dphdy * g_rph + dthdy * dphdy * g_thph  )
			g_yz =    drdy *  drdz * g_rr + dthdy * dthdz * g_thth + dphdy * dphdz * g_phph + (  drdy*dthdz +  drdz*dthdy ) * g_rth + (  drdy*dphdz +  drdz*dphdy ) * g_rph + ( dthdy*dphdz + dthdz*dphdy ) * g_thph
			g_zz =    drdz *  drdz * g_rr + dthdz * dthdz * g_thth + dphdz * dphdz * g_phph + 2.0 * (    drdz * dthdz * g_rth +  drdz * dphdz * g_rph + dthdz * dphdz * g_thph  ) 

		elif rho < 1.0e-7 and r > 1.0e-7: # on axis but not origin

			#if ( ltrace5 ) {							#What is ltrace5??
			#     print(  " coord transforming on axis but not origin ... /n"
			#     print(  " x = " << xx
			#     print(  " y = " << yy
			#     print(  " z = " << zz
			#     print(  " rho = " <<rho
			#     print(  " r  = " << r
			#}
			
			rho_axis = sqrt( rho_sqrd + epsilon_sq )

			g_xx =    g_bare_rr - ( yy*yy + zz*zz ) * ( g_bare_rr - g_bare_thth ) / (r*r) - yy*yy * ( g_bare_thth - g_bare_phph ) / (rho_axis*rho_axis) + 2.0/r * (xx/rho_axis) * ( xx*zz/r * g_bare_rth - yy * g_bare_rph - zz*(yy/rho_axis) * g_thph  )
			g_xy =   xx*yy/(r*r) * ( g_bare_rr - g_bare_phph ) + xx*yy/(rho_axis*rho_axis) * zz*zz/(r*r) * ( g_bare_thth - g_bare_phph ) + 2.0 * zz/(r*r) * (xx/rho_axis) * yy * g_bare_rth + (xx/rho_axis) * xx * g_bare_rph / r - (yy/rho_axis) * yy * g_bare_rph / r + pow((xx/rho_axis),2) * zz * g_bare_thph / r - pow((yy/rho_axis),2) * zz * g_bare_thph / r
			g_xz =    xx*zz/(r*r) * ( g_bare_rr - g_bare_thth ) + (   zz*zz * (xx/rho_axis) * g_bare_rth - xx*rho * g_bare_rth ) / (r*r) - (zz/r) * (yy/rho_axis) * g_bare_rph + yy * g_bare_thph / r 
			g_yy =    g_bare_rr - (xx*xx + zz*zz)/(r*r) * ( g_bare_rr - g_bare_thth ) - pow((xx/rho_axis),2) * ( g_bare_thth - g_bare_phph ) + 2.0/r * (yy/rho_axis) * ( yy*zz/r * g_bare_rth + (xx/rho_axis)*zz * g_bare_thph + xx * g_bare_rph )
			g_yz =    yy*zz/(r*r) * ( g_bare_rr - g_bare_thth ) + (yy/rho_axis)*zz*zz/(r*r) * g_bare_rth - yy*rho/(r*r) * g_bare_rth + (zz/r) * (xx/rho_axis) * g_bare_rph - xx * g_bare_thph / r
			g_zz =   g_bare_rr - (rho*rho)/(r*r) * ( g_bare_rr - g_bare_thth ) - 2.0 * zz * rho * g_bare_rth / (r*r)

		elif rho < 1.0e-7 and r < 1.0e-7:  # at the origin
			#   if ( ltrace5 ) {
			#   	print( " performing transformation at the origin ... "
			#   	std::cout<< " x = " << xx
			#   	print( " y = " << yy
			#   	print( " z = " << zz
			#   	print( " rho = " << rho
			#   	print( "  r  = " << r
			#   }
			rho_axis = sqrt( rho_sqrd + epsilon_sq )
			r_origin = sqrt( r_sqrd + epsilon_sq )


			g_xx = g_bare_rr - ( yy*yy + zz*zz ) / (pow(r_origin,2)) * ( g_bare_rr - g_bare_thth ) - yy*yy / (rho_axis*rho_axis) * ( g_bare_thth - g_bare_phph ) + 2.0/(r_origin) * (xx/rho_axis) * ( xx*zz/r_origin * g_bare_rth - yy * g_bare_rph - zz*(yy/rho_axis) * g_thph  )
			g_xy =   xx*yy/(pow(r_origin,2)) * ( g_bare_rr - g_bare_phph ) + xx*yy/(rho_axis*rho_axis) * zz*zz/(pow(r_origin,2)) * ( g_bare_thth - g_bare_phph ) + 2.0 * zz/(pow(r_origin,2)) * (xx/rho_axis) * yy * g_bare_rth + (xx/rho_axis) * (xx/r_origin) * g_bare_rph - (yy/rho_axis) * (yy/r_origin) * g_bare_rph + pow((xx/rho_axis),2) * (zz/r_origin) * g_bare_thph - pow((yy/rho_axis),2) * (zz/r_origin) * g_bare_thph
			g_xz =   xx*zz/(pow(r_origin,2)) * ( g_bare_rr - g_bare_thth ) + (   pow((zz/r_origin),2) * (xx/rho_axis) * g_bare_rth - (xx/r_origin)*(rho/r_origin) * g_bare_rth ) - (zz/r_origin) * (yy/rho_axis) * g_bare_rph + (yy/r_origin) * g_bare_thph
			g_yy =   g_bare_rr - ( pow((xx/r_origin),2) + pow((zz/r_origin),2)) * ( g_bare_rr - g_bare_thth ) - pow((xx/rho_axis),2) * ( g_bare_thth - g_bare_phph ) + 2.0 * (yy/rho_axis) * (  (yy/r_origin)*(zz/r_origin) * g_bare_rth + (xx/rho_axis)*(zz/r_origin) * g_bare_thph + (xx/r_origin) * g_bare_rph )
			g_yz =    (yy/r_origin)*(zz/r_origin) * ( g_bare_rr - g_bare_thth ) + (yy/rho_axis)*pow((zz/r_origin),2) * g_bare_rth - (yy/r_origin)*(rho/r_origin) * g_bare_rth + (zz/r_origin) * (xx/rho_axis) * g_bare_rph - (xx/r_origin) * g_bare_thph
			g_zz =    g_bare_rr - pow((rho/r_origin),2) * ( g_bare_rr - g_bare_thth ) - 2.0 * (zz/r_origin) * (rho/r_origin) * g_bare_rth
		else:
			print(" something strange happened ... ")
			
		#Checking to see if the determinant is negative
		determinant = g_rr*(g_thth*g_phph-g_thph*g_thph)-g_rth*(g_rth*g_phph-g_rph*g_rth)+g_rph*(g_rth*g_thph-g_thth*g_rph)
		#print( "the determinant of the metric is given by "<<determinant<<std::endl
		#printing out each component
		#print("g_rr= "<<g_rr<< "g_rth= " <<g_rth <<"g_rph= " <<g_rph <<"g_thth= " <<g_thth<<"g_thph= "<<g_thph <<"g_phph= "<<g_phph<<std::endl
		if determinant<0:
			print("The determinant(for the spherical metric) = " + determinant)
			print(" x = " + xx)
			print(" y = " + yy)
			print(" z = " + zz)
			print(" g_bare_rr = " + g_bare_rr)    
			print(" g_bare_rth = " + g_bare_rth) 
			print(" g_bare_rph = " + g_bare_rph)  
			print(" g_bare_thth = " + g_bare_thth)    
			print(" g_bare_thph = " + g_bare_thph)    
			print(" g_bare_phph = " + g_bare_phph)   
				#print(" grid_spacing = " << grid_spacing
			print(" rho_axis = " + rho_axis)
			print(" r_origin = " + r_origin)
			print(" epsilon_sq = " + epsilon_sq)
			print(" A_lm = " + A_lm)
			print(" B_lm = " + B_lm)
			print(" C_lm = " + C_lm) 
			print(" h_bare_rr = " + h_bare_rr)    
			print(" h_bare_rth = " + h_bare_rth) 
			print(" h_bare_rph = " + h_bare_rph)  
			print(" h_bare_thth = " + h_bare_thth)    
			print(" h_bare_thph = " + h_bare_thph)    
			print(" h_bare_phph = " + h_bare_phph) 
			print(" r = " + r)
		# if ( ltrace5 ) {
		#	if ( abs(g_xx) > 2.0
		#	     || abs(g_xy) > 2.0
		#	     || abs(g_xz) > 2.0
		#	     || abs(g_yy) > 2.0
		#	     || abs(g_yz) > 2.0
		#	     || abs(g_zz) > 2.0 )  {
		#		print(" a metric function is larger than expected "
		#		print(" x = "<< xx
		#		print(" y = "<< yy
		#		print(" z = " <<zz
		#		print(" g_xx = "<< g_xx
		#		print(" g_xy = "<< g_xy
		#		print(" g_xz = "<< g_xz
		#		print(" g_yy = "<<g_yy
		#		print(" g_yz = " << g_yz
		#		print(" g_zz = " << g_zz
		#		print(" drdx = " << drdx
		#		print(" drdy = " << drdy
		#		print(" drdz = " << drdz
		#		print(" dthdx = " << dthdx
		#		print(" dthdy = " << dthdy
		#		print(" dthdz = " << dthdz
		#		print(" dphdx = " << dphdx
		#		print(" dphdy = " << dphdy
		#		print(" dphdz = " << dphdz
		#		print(" g_bare_rr = " << g_bare_rr
		#		print(" g_bare_rth = " << g_bare_rth
		#		print(" g_bare_rph = " << g_bare_rph
		#		print(" g_bare_thth = " << g_bare_thth
		#		print(" g_bare_thph = " << g_bare_thph
		#		print(" g_bare_phph = " << g_bare_phph
		#		print(" grid_spacing = " << grid_spacing
		#		print(" rho_axis = " << rho_axis
		#		print(" r_origin = " << r_origin
		#		print(" epsilon_sq = " << epsilon_sq
		#		print(" A_2m = " << A_2m
		#		print(" B_2m = " << B_2m
		#		print(" C_2m = " << C_2m
		#	}
		#
										#I believe this is no longer
		#	if (     (i == (nx-1)/4 && j == (ny-1)/4 && k == (nz-1)/4)	#necessary
		#	     ||(i == (nx-1)/2 && j == (ny-1)/2 && k == 3*(nz-1)/4)
		#	     ||(i == (nx-1)/2 && j == (ny-1)/2 && k == (nz-1)/2)
		#	    ) {

		#		print(" i = " << i  
		#		print(" j = " << j  
		#		print(" k = " << k 
		#		print(" x = " << x  
		#		print(" y = " << y  
		#		print(" z = " << z  
		#		print(" drdx = " << drdx  
		#		print(" drdy = " << drdy  
		#		print(" drdz = " << drdz  
		#		print(" dthdx = " << dthdx  
		#		print(" dthdy = " << dthdy  
		#		print(" dthdz = " << dthdz  
		#		print(" dphdx = " << dphdx  
		#		print(" dphdy = " << dphdy  
		#		print(" dphdz = " << dphdz  
		#		print(" g_xx = " << g_xx   
		#		print(" g_xy = " << g_xy   
		#		print(" g_xz = " << g_xz  
		#		print(" g_yy = " << g_yy   
		#		print(" g_yz = " << g_yz   
		#		print(" g_zz = " << g_zz   
		#		print(" g_bare_rr = " << g_bare_rr <<std::endl    
		#		print(" g_bare_rth = " << g_bare_rth<<std::endl 
		#		print(" g_bare_rph = " << g_bare_rph<<std::endl  
		#		print(" g_bare_thth = " << g_bare_thth<<std::endl    
		#		print(" g_bare_thph = " << g_bare_thph<<std::endl    
		#		print(" g_bare_phph = " << g_bare_phph<<std::endl   
		#		print(" grid_spacing = " << grid_spacing
		#		print(" rho_axis = " << rho_axis<<std::endl
		#		print(" r_origin = " << r_origin<<std::endl
		#		print(" epsilon_sq = " << epsilon_sq<<std::endl
		#		print(" A_2m = " << A_2m<<std::endl
		#		print(" B_2m = " << B_2m<<std::endl
		#		print(" C_2m = " << C_2m<<std::endl 
		#		print(" h_bare_rr = " << h_bare_rr <<std::endl    
		#		print(" h_bare_rth = " << h_bare_rth<<std::endl 
		#		print(" h_bare_rph = " << h_bare_rph<<std::endl  
		#		print(" h_bare_thth = " << h_bare_thth<<std::endl    
		#		print(" h_bare_thph = " << h_bare_thph<<std::endl    
		#		print(" h_bare_phph = " << h_bare_phph<<std::endl 
		#		print(" r = "<< r<<std::endl
		#	}
		#}

		# printing out the determinant of the metric
		#minkowski metric
		#double determinant = g_xx*(g_yy*g_zz-g_yz*g_yz)-g_xy*(g_xy*g_zz-g_xz*g_xy)+g_xz*(g_xy*g_yz-g_yy*g_xz)
		#preturbed metric
		#double determinant = g_rr*(g_thth*g_phph-g_thph*g_thph)-g_rth*(g_rth*g_phph-g_rph*g_rth)+g_rph*(g_rth*g_thph-g_thth*g_rph)
		#print( "the determinant of the metric is given by "<<determinant<<std::endl
		#printing out each component
		#print("g_rr= "<<g_rr<< "g_rth= " <<g_rth <<"g_rph= " <<g_rph <<"g_thth= " <<g_thth<<"g_thph= "<<g_thph <<"g_phph= "<<g_phph<<std::endl

		#U_ALPHA  = 1.0
		#U_BETA0 = 0.0
		#U_BETA1 = 0.0
		#U_BETA2 = 0.0

		#U_GAUGEB0 = 0.0
		#U_GAUGEB1 = 0.0
		#U_GAUGEB2 = 0.0

		#U_CAP_GT0 = 0.0
		#U_CAP_GT1 = 0.0
		#U_CAP_GT2 = 0.0

		#U_GT00 = g_xx
		U_GT01[xc,yc] = g_xy
		#U_GT02 = g_xz
		#U_GT11 = g_yy
		#U_GT12 = g_yz
		#U_GT22 = g_zz
		#U_AT00 =  0.0
		#U_AT01 =  0.0
		#U_AT02 =  0.0
		#U_AT11 =  0.0
		#U_AT12 =  0.0
		#U_AT22 =  0.0

		#U_CHI   = 1.0
		#U_TRK   = 0.0
		#	var[VAR::U_THETA] = 0.0

		#	var[VAR::U_FLAG] = 0.0
		#	var[VAR::U_TUU00] = 0.0
		#	var[VAR::U_TUU01] = 0.0
		#	var[VAR::U_TUU02] = 0.0
		#	var[VAR::U_TUU03] = 0.0
		#	var[VAR::U_TUU11] = 0.0
		#	var[VAR::U_TUU12] = 0.0
		#	var[VAR::U_TUU13] = 0.0
		#	var[VAR::U_TUU22] = 0.0
		#	var[VAR::U_TUU23] = 0.0
		#	var[VAR::U_TUU33] = 0.0

X,Y = np.meshgrid(x,y,indexing='ij')

fig = plt.figure(1)
ax = fig.add_subplot(111,projection='3d')
surf = ax.plot_surface(X,Y,U_GT01, cmap=cm.viridis)
plt.xlabel('x')
plt.ylabel('y')
fig.colorbar(surf)

print("finished")

        

finished
